In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.1
seed = 0
alpha = 10e-3
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.16666667 0.16666667 0.1        0.         0.
  0.14285714 0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.16666667 0.         0.16666667 0.1        0.         0.
  0.         0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16666667]
 [0.16666667 0.16666667 0.         0.         0.         0.
  0.14285714 0.         0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.16666667 0.        ]
 [0.1        0.1        0.         0.         0.1        0.1
  0.1        0.1        0.1        0.         0.         0.1
  0.         0.         0.         0.         0.         0.
  0.         0.1       ]
 [0.         0.         0.         0.1        0.         0.14285714
  0.         0.14285714 0.14285714 0.         0.         0.
  0.         0.14285714 0.14285714 0.         0.  

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[5, 5, 5, 5, 9, 6, 3, 6, 5, 6, 1, 6, 5, 6, 5, 5, 3, 2, 6, 4]
2
4
7
9
10


In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0, 5.0, 2.0])
W2 = np.array([2.0, 2.0, 0.0, 5.0, 2.0, 3.0, -5.0, -2.0])
W = [W1, W2]
m = 200
n = 8
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(8, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)

        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss.detach(), parameters_to_vector(model.parameters()).detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(92.1181) tensor([ 0.0418, -0.1291,  0.0289,  0.2754,  0.3229, -0.0085,  0.3124,  0.0311])
0 tensor(68.4036) tensor([ 0.0835, -0.0312,  0.0022,  0.3770,  0.3478,  0.0709,  0.1856, -0.0265])
0 tensor(46.1244) tensor([ 0.1528,  0.0255, -0.0159,  0.4527,  0.3682,  0.1260,  0.0969, -0.0847])
0 tensor(78.2053) tensor([ 0.1408,  0.0448, -0.0105,  0.5125,  0.3762,  0.1760,  0.0058, -0.0918])
1 tensor(77.8459) tensor([ 0.2132,  0.0795, -0.0257,  0.6221,  0.4206,  0.2121, -0.0910, -0.2028])
1 tensor(57.7171) tensor([ 0.2514,  0.1686, -0.0492,  0.7160,  0.4445,  0.2850, -0.2075, -0.2549])
1 tensor(39.4508) tensor([ 0.3148,  0.2201, -0.0650,  0.7860,  0.4638,  0.3361, -0.2889, -0.3072])
1 tensor(65.9270) tensor([ 0.3036,  0.2373, -0.0595,  0.8420,  0.4719,  0.3823, -0.3733, -0.3120])
2 tensor(65.8242) tensor([ 0.3699,  0.2683, -0.0728,  0.9429,  0.5133,  0.4153, -0.4626, -0.4127])
2 tensor(48.7315) tensor([ 0.4050,  0.3495, -0.0934,  1.0297,  0.5360,  0.4823, -0.5697, -0.4599])
2 tensor(3

21 tensor(2.4742) tensor([ 1.7383,  1.8094, -0.2544,  4.0731,  1.5688,  2.3662, -3.9921, -1.9624])
22 tensor(2.6555) tensor([ 1.7499,  1.8117, -0.2534,  4.0925,  1.5787,  2.3722, -4.0112, -1.9757])
22 tensor(1.9518) tensor([ 1.7559,  1.8241, -0.2525,  4.1110,  1.5863,  2.3855, -4.0319, -1.9820])
22 tensor(1.8554) tensor([ 1.7654,  1.8305, -0.2517,  4.1250,  1.5920,  2.3971, -4.0458, -1.9860])
22 tensor(2.1164) tensor([ 1.7620,  1.8316, -0.2484,  4.1391,  1.5958,  2.4070, -4.0637, -1.9799])
23 tensor(2.2798) tensor([ 1.7726,  1.8335, -0.2473,  4.1570,  1.6050,  2.4125, -4.0814, -1.9920])
23 tensor(1.6778) tensor([ 1.7782,  1.8449, -0.2461,  4.1741,  1.6122,  2.4249, -4.1006, -1.9976])
23 tensor(1.6191) tensor([ 1.7868,  1.8506, -0.2452,  4.1871,  1.6175,  2.4357, -4.1134, -2.0010])
23 tensor(1.8119) tensor([ 1.7836,  1.8516, -0.2420,  4.2003,  1.6211,  2.4449, -4.1301, -1.9951])
24 tensor(1.9589) tensor([ 1.7933,  1.8531, -0.2408,  4.2167,  1.6297,  2.4501, -4.1465, -2.0059])
24 tensor(

44 tensor(0.0874) tensor([ 1.9703,  2.0003, -0.1106,  4.8193,  1.9005,  2.8466, -4.7986, -2.0612])
44 tensor(0.1052) tensor([ 1.9715,  2.0008, -0.1095,  4.8220,  1.9019,  2.8496, -4.8010, -2.0607])
44 tensor(0.0850) tensor([ 1.9705,  2.0007, -0.1083,  4.8254,  1.9029,  2.8520, -4.8049, -2.0582])
45 tensor(0.0973) tensor([ 1.9722,  2.0002, -0.1072,  4.8284,  1.9049,  2.8530, -4.8085, -2.0590])
45 tensor(0.0767) tensor([ 1.9730,  2.0017, -0.1056,  4.8317,  1.9067,  2.8556, -4.8121, -2.0595])
45 tensor(0.0928) tensor([ 1.9741,  2.0021, -0.1046,  4.8343,  1.9080,  2.8584, -4.8143, -2.0591])
45 tensor(0.0742) tensor([ 1.9732,  2.0019, -0.1034,  4.8374,  1.9090,  2.8607, -4.8179, -2.0567])
46 tensor(0.0850) tensor([ 1.9747,  2.0015, -0.1024,  4.8402,  1.9108,  2.8616, -4.8213, -2.0574])
46 tensor(0.0673) tensor([ 1.9754,  2.0028, -0.1009,  4.8432,  1.9126,  2.8640, -4.8246, -2.0578])
46 tensor(0.0819) tensor([ 1.9764,  2.0032, -0.0999,  4.8456,  1.9138,  2.8667, -4.8266, -2.0574])
46 tensor(

69 tensor(0.0049) tensor([ 1.9966,  2.0057, -0.0320,  4.9691,  1.9802,  2.9654, -4.9628, -2.0215])
69 tensor(0.0034) tensor([ 1.9964,  2.0056, -0.0317,  4.9698,  1.9804,  2.9660, -4.9635, -2.0208])
70 tensor(0.0039) tensor([ 1.9966,  2.0054, -0.0313,  4.9702,  1.9808,  2.9662, -4.9642, -2.0207])
70 tensor(0.0034) tensor([ 1.9967,  2.0055, -0.0308,  4.9707,  1.9812,  2.9667, -4.9648, -2.0207])
70 tensor(0.0043) tensor([ 1.9968,  2.0055, -0.0304,  4.9712,  1.9814,  2.9674, -4.9651, -2.0204])
70 tensor(0.0030) tensor([ 1.9966,  2.0054, -0.0301,  4.9718,  1.9817,  2.9679, -4.9658, -2.0198])
71 tensor(0.0035) tensor([ 1.9969,  2.0052, -0.0297,  4.9722,  1.9820,  2.9681, -4.9665, -2.0197])
71 tensor(0.0030) tensor([ 1.9969,  2.0053, -0.0292,  4.9727,  1.9823,  2.9685, -4.9670, -2.0197])
71 tensor(0.0038) tensor([ 1.9970,  2.0053, -0.0289,  4.9731,  1.9826,  2.9692, -4.9673, -2.0194])
71 tensor(0.0026) tensor([ 1.9968,  2.0052, -0.0286,  4.9737,  1.9828,  2.9697, -4.9680, -2.0188])
72 tensor(

93 tensor(0.0002) tensor([ 1.9990,  2.0019, -0.0089,  4.9942,  1.9957,  2.9913, -4.9921, -2.0059])
94 tensor(0.0002) tensor([ 1.9990,  2.0019, -0.0088,  4.9942,  1.9958,  2.9914, -4.9923, -2.0058])
94 tensor(0.0002) tensor([ 1.9990,  2.0019, -0.0086,  4.9943,  1.9958,  2.9915, -4.9924, -2.0058])
94 tensor(0.0002) tensor([ 1.9991,  2.0019, -0.0085,  4.9944,  1.9959,  2.9917, -4.9925, -2.0057])
94 tensor(0.0002) tensor([ 1.9990,  2.0018, -0.0084,  4.9945,  1.9959,  2.9918, -4.9926, -2.0055])
95 tensor(0.0002) tensor([ 1.9991,  2.0018, -0.0083,  4.9946,  1.9960,  2.9919, -4.9927, -2.0055])
95 tensor(0.0002) tensor([ 1.9991,  2.0018, -0.0082,  4.9947,  1.9961,  2.9920, -4.9928, -2.0055])
95 tensor(0.0002) tensor([ 1.9991,  2.0018, -0.0081,  4.9948,  1.9961,  2.9921, -4.9929, -2.0054])
95 tensor(0.0002) tensor([ 1.9991,  2.0017, -0.0080,  4.9949,  1.9962,  2.9923, -4.9931, -2.0052])
96 tensor(0.0002) tensor([ 1.9991,  2.0017, -0.0079,  4.9950,  1.9963,  2.9923, -4.9932, -2.0052])
96 tensor(

In [11]:
parameters_to_vector(model.parameters())

tensor([ 1.9992,  2.0014, -0.0064,  4.9962,  1.9971,  2.9939, -4.9947, -2.0042],
       grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-2
                                   )

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [15]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [16]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 0.0832,  0.3462,  0.0457, -0.0490, -0.3360, -0.2969, -0.0429,  0.1398])


tensor([ 0.1664,  0.6923,  0.0914, -0.0981, -0.6720, -0.5937, -0.0858,  0.2795],
       grad_fn=<CatBackward0>)

In [17]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:00<04:18,  7.73it/s]

Training_loss 69.91521,   Relative Error 0.95995


  0%|          | 2/2000 [00:00<04:32,  7.33it/s]

Training_loss 68.38413,   Relative Error 0.94939


  0%|          | 3/2000 [00:00<04:11,  7.92it/s]

...

Training_loss 0.00436,   Relative Error 0.00822
Training_loss 0.00434,   Relative Error 0.00820
Training_loss 0.00438,   Relative Error 0.00825


100%|█████████▉| 1999/2000 [04:31<00:00,  9.86it/s]

Training_loss 0.00432,   Relative Error 0.00818
Training_loss 0.00427,   Relative Error 0.00813
Training_loss 0.00428,   Relative Error 0.00815


100%|██████████| 2000/2000 [04:31<00:00,  7.38it/s]

Training_loss 0.00430,   Relative Error 0.00816


In [18]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([ 1.9969,  2.0082, -0.0050,  4.9995,  1.9717,  2.9688, -4.9455, -1.9878],
       grad_fn=<CatBackward0>)

In [19]:
for j in G.neighbors(0):
    print(j)

2
4
7
9
10


In [22]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [23]:
np.save( 'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), total_rel_error)

In [24]:
'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'training_loss_dfedu0_01_pout0_1'

In [25]:
'relative_error_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'relative_error_dfedu0_01_pout0_1'